In [1]:
import sys
sys.path.append("../../rcsc")

import os
import pickle
import random

import pandas as pd
import numpy as np

from tqdm import tqdm

from time import sleep

import json

from functions import df_tbca, codigos_tbca, alimentos_gemini, alimentos_gbt, alimentos_deepseek, codigos, mapa, gemini_diets, gbt_diets, deepseek_diets

O arquivo final 'dataframe_nutricional_tbca.pkl' já existe.
Para gerar novamente, apague o arquivo manualmente.

DataFrame carregado com sucesso!
Total de alimentos no DataFrame: 5658
O arquivo 'dados_completos_tbca.pkl' já existe.
O web scraping não será executado novamente.

Verificação: 5687 registros já estão salvos.
Exemplo do primeiro registro: ('BRC0001C', 'Abacate, polpa, in natura, Brasil', 'Frutas e derivados')


In [5]:
gemini_diets[0]["1"]

{'Café da Manhã': [{'alimento': 'Leite Integral', 'quantidade': '100'},
  {'alimento': 'Aveia em Flocos', 'quantidade': '30'},
  {'alimento': 'Maçã', 'quantidade': '130'}],
 'Lanche da Manhã': [{'alimento': 'Banana Prata', 'quantidade': '100'},
  {'alimento': 'Amendoim torrado sem sal', 'quantidade': '10'}],
 'Almoço': [{'alimento': 'Arroz Cozido', 'quantidade': '100'},
  {'alimento': 'Feijão Cozido', 'quantidade': '80'},
  {'alimento': 'Peito de Frango Grelhado', 'quantidade': '110'},
  {'alimento': 'Salada de Folhas e Tomate', 'quantidade': '150'},
  {'alimento': 'Azeite para tempero', 'quantidade': '5'}],
 'Lanche da Tarde': [{'alimento': 'Pão Francês', 'quantidade': '50'},
  {'alimento': 'Queijo Minas Padrão', 'quantidade': '40'}],
 'Jantar': [{'alimento': 'Batata Doce Cozida', 'quantidade': '150'},
  {'alimento': 'Ovos Cozidos', 'quantidade': '80'},
  {'alimento': 'Salada de Brócolis e Cenoura Cozida', 'quantidade': '150'}],
 'Ceia': [{'alimento': 'Chá de Ervas (sem açúcar)', 'qua

In [7]:


def calcular_nutrientes_dieta(dieta_dia: dict, 
                              mapa_codigos: dict, 
                              df_nutricional: pd.DataFrame, 
                              atributos_desejados: list) -> dict:
    """
    Calcula a soma total de nutrientes para um dia de dieta.

    Args:
        dieta_dia (dict): Dicionário representando o plano alimentar do dia.
        mapa_codigos (dict): Dicionário que mapeia nomes de alimentos para códigos TBCA.
        df_nutricional (pd.DataFrame): DataFrame com informações nutricionais detalhadas dos alimentos.
        atributos_desejados (list): Lista de strings com os nomes dos nutrientes a serem somados.

    Returns:
        dict: Um dicionário com a soma total de cada atributo solicitado para a dieta fornecida.
    """
    # Inicializa o dicionário de totais com zero para cada atributo
    soma_nutrientes = {atributo: 0.0 for atributo in atributos_desejados}
    
    # Para buscas rápidas, é muito mais eficiente definir a coluna 'codigo' como o índice do DataFrame
    if 'codigo' in df_nutricional.columns:
        df_indexed = df_nutricional.set_index('codigo')
    else:
        print("Erro: A coluna 'codigo' não foi encontrada no DataFrame.")
        return soma_nutrientes

    # Itera sobre cada refeição no dia (ex: "Café da Manhã", "Almoço")
    for refeicao_nome, itens_refeicao in dieta_dia.items():
        # Itera sobre cada alimento dentro da refeição
        for item in itens_refeicao:
            nome_alimento = item.get('alimento')
            quantidade_str = item.get('quantidade')

            # Pula se o alimento não estiver no mapa de códigos
            if nome_alimento not in mapa_codigos:
                print(f"Aviso: Alimento '{nome_alimento}' não encontrado no mapa de códigos. Ignorando.")
                continue

            codigos_str = mapa_codigos[nome_alimento]
            
            # Pula se o código for um marcador de item a ser ignorado (ex: '-')
            if codigos_str == '-':
                continue

            try:
                quantidade_total_g = float(quantidade_str)
            except (ValueError, TypeError):
                print(f"Aviso: Quantidade inválida ('{quantidade_str}') para '{nome_alimento}'. Ignorando.")
                continue

            # Divide os códigos se houver mais de um (ex: "codigo1,codigo2")
            lista_de_codigos = codigos_str.split(',')
            num_codigos = len(lista_de_codigos)
            
            # Divide a porção igualmente entre os alimentos componentes
            porcao_por_codigo = quantidade_total_g / num_codigos

            for codigo in lista_de_codigos:
                codigo = codigo.strip()
                try:
                    # Busca os dados do alimento no DataFrame indexado (muito rápido)
                    dados_alimento = df_indexed.loc[codigo]
                    
                    # Soma os valores para cada atributo desejado
                    for atributo in atributos_desejados:
                        if atributo in dados_alimento and pd.notna(dados_alimento[atributo]):
                            valor_por_100g = dados_alimento[atributo]
                            # Calcula o valor do nutriente na porção consumida
                            valor_na_porcao = (valor_por_100g / 100.0) * porcao_por_codigo
                            soma_nutrientes[atributo] += valor_na_porcao

                except KeyError:
                    print(f"Aviso: Código '{codigo}' para '{nome_alimento}' não encontrado no DataFrame. Ignorando.")
                    continue
    
    return soma_nutrientes

# --- Exemplo de Uso ---

# 1. DADOS DE ENTRADA (Simulando os seus dados)

# X: Dieta do dia
dieta_exemplo = gemini_diets[0]["1"]

# Y: Mapa de nomes para códigos
mapa_codigos_exemplo = mapa
# Z: DataFrame de nutrientes (amostra com dados fictícios para o exemplo funcionar)
df_exemplo = df_tbca

# W: Lista de atributos a serem somados
atributos_para_somar = [
    "Energia",
    'Proteína',
    'Lipídios',
    'Carboidrato disponível',
    'Fibra alimentar',
    'Cálcio',
    'Ferro',
    'Vitamina A (RAE)'
]

# 2. CHAMADA DA FUNÇÃO
totais_dieta = calcular_nutrientes_dieta(dieta_exemplo, mapa_codigos_exemplo, df_exemplo, atributos_para_somar)

# 3. EXIBIÇÃO DO RESULTADO
print("\n--- TOTAIS DE NUTRIENTES PARA A DIETA ---")
if totais_dieta:
    for nutriente, total in totais_dieta.items():
        # Arredonda para 2 casas decimais para melhor visualização
        print(f"{nutriente}: {total:.2f}")


--- TOTAIS DE NUTRIENTES PARA A DIETA ---
Energia: 1438.50
Proteína: 80.42
Lipídios: 42.00
Carboidrato disponível: 170.95
Fibra alimentar: 27.58
Cálcio: 649.09
Ferro: 9.52
Vitamina A (RAE): 1809.85
